# Session 3 - Maîtriser l'Attention

**Module** : Réseaux de Neurones Approfondissement  
**Durée** : 2h  
**Objectif** : Implémenter l'attention de A à Z et découvrir le Multi-Head

---

## Objectifs pédagogiques

À la fin de cette session, vous serez capable de :
1. Implémenter la fonction `scaled_dot_product_attention()` complète
2. Créer une classe `SelfAttention` réutilisable en PyTorch
3. Visualiser et interpréter l'attention d'un vrai modèle (CamemBERT)
4. Comprendre pourquoi on utilise **plusieurs têtes** d'attention

---

## Rappel : Où en sommes-nous ?

Dans la session précédente, vous avez :
- Compris le **Positional Encoding** (encoder l'ordre des mots)
- Vu le lien entre **similarité** et **produit scalaire**
- Découvert les concepts de **Query, Key, Value**
   - Q : Ce que je cherche
   - K : Les étiquettes
   - V : Le contenu
- Calculé les scores d'attention étape par étape

Aujourd'hui, on passe à l'**implémentation complète** !

## 0. Installation et imports

In [ ]:
# Installation des dépendances (Google Colab)
!pip install torch matplotlib numpy transformers bertviz -q

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import math

torch.manual_seed(42)
print(f"PyTorch version: {torch.__version__}")

PyTorch version: 2.10.0


---

## 1. Rappel : La formule de l'attention

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

| Étape | Opération | Rôle |
|-------|-----------|------|
| 1 | $QK^T$ | Calculer les scores de similarité |
| 2 | $\div \sqrt{d_k}$ | Stabiliser les gradients |
| 3 | softmax | Transformer en probabilités |
| 4 | $\times V$ | Moyenne pondérée des values |

Dans la session précédente, vous avez fait ces étapes **séparément**. Maintenant, on les **combine** dans une fonction.

---

## 2. Exercice 1 : Fonction d'attention complète

Regroupez les 4 étapes dans une seule fonction réutilisable.

In [5]:
def scaled_dot_product_attention(Q, K, V):
    """
    Calcule le Scaled Dot-Product Attention.
    
    Args:
        Q: Queries, shape (..., seq_len, d_k)
        K: Keys, shape (..., seq_len, d_k)
        V: Values, shape (..., seq_len, d_v)
    
    Returns:
        output: Résultat de l'attention, même shape que V
        attention_weights: Poids d'attention, shape (..., seq_len, seq_len)
    """
    # CORRECTION 1: Récupérer d_k (dernière dimension de K)
    d_k = K.shape[-1]
    
    # CORRECTION 2: Calculer les scores QK^T
    scores = Q @ K.transpose(-2, -1)
    
    # CORRECTION 3: Appliquer le scaling (diviser par sqrt(d_k))
    scaled_scores = scores / math.sqrt(d_k)
    
    # CORRECTION 4: Appliquer softmax sur la dernière dimension
    attention_weights = F.softmax(scaled_scores, dim=-1)
    
    # CORRECTION 5: Calculer la sortie (weights @ V)
    output = attention_weights @ V
    
    return output, attention_weights

In [6]:
# Test de votre fonction
Q_test = torch.randn(4, 8)  # 4 tokens, dimension 8
K_test = torch.randn(4, 8)
V_test = torch.randn(4, 8)

output, weights = scaled_dot_product_attention(Q_test, K_test, V_test)

print(f"Output shape: {output.shape}")    # Attendu: (4, 8)
print(f"Weights shape: {weights.shape}")  # Attendu: (4, 4)
print(f"Somme des poids par ligne: {weights.sum(dim=-1)}")  # Attendu: [1, 1, 1, 1]

Output shape: torch.Size([4, 8])
Weights shape: torch.Size([4, 4])
Somme des poids par ligne: tensor([1.0000, 1.0000, 1.0000, 1.0000])


### Vérification : Effet du scaling

Pourquoi diviser par $\sqrt{d_k}$ ? Voyons l'effet.

In [7]:
# Comparaison avec et sans scaling
d_k_grand = 512  # Dimension typique dans un Transformer

Q_grand = torch.randn(10, d_k_grand)
K_grand = torch.randn(10, d_k_grand)

# Sans scaling
scores_sans = Q_grand @ K_grand.T
poids_sans = F.softmax(scores_sans, dim=-1)

# Avec scaling
scores_avec = (Q_grand @ K_grand.T) / math.sqrt(d_k_grand)
poids_avec = F.softmax(scores_avec, dim=-1)

print("=== SANS SCALING ===")
print(f"Scores - min: {scores_sans.min():.1f}, max: {scores_sans.max():.1f}")
print(f"Poids d'attention max par ligne: {poids_sans.max(dim=-1).values[:5]}")

print("\n=== AVEC SCALING ===")
print(f"Scores - min: {scores_avec.min():.1f}, max: {scores_avec.max():.1f}")
print(f"Poids d'attention max par ligne: {poids_avec.max(dim=-1).values[:5]}")

print("\nAvec scaling, les poids sont mieux répartis (pas de valeur proche de 1)")

=== SANS SCALING ===
Scores - min: -59.2, max: 58.5
Poids d'attention max par ligne: tensor([1.0000, 0.9968, 1.0000, 1.0000, 1.0000])

=== AVEC SCALING ===
Scores - min: -2.6, max: 2.6
Poids d'attention max par ligne: tensor([0.5353, 0.2391, 0.3317, 0.2887, 0.4791])

Avec scaling, les poids sont mieux répartis (pas de valeur proche de 1)


---

## 3. Exercice 2 : Classe SelfAttention

### D'où viennent Q, K, V ?

Jusqu'ici, on a utilisé des tenseurs aléatoires. En pratique, **Q, K, V sont calculés à partir des embeddings** via des matrices apprenables.

```
x (embeddings) ──┬──► W_q ──► Q   (ce que je cherche)
                 ├──► W_k ──► K   (comment je me présente)
                 └──► W_v ──► V   (l'info que je transmets)
```

### Pourquoi 3 matrices différentes ?

Si on utilisait directement `x @ x.T`, on calculerait les **similarités sémantiques** entre mots. "Pikachu" serait attentif à "Raichu", "électrique"...

Avec des projections différentes (W_q, W_k, W_v), le modèle peut capturer d'autres types de relations : **syntaxiques** (sujet-verbe), **contextuelles**, etc.

Les matrices sont **apprises** pendant l'entraînement : elles s'ajustent pour que la formule `softmax(Q @ K.T) @ V` produise des résultats utiles pour la tâche.

> **Pour approfondir** : voir la note *"Comprendre le Mécanisme d'Attention"* qui détaille le rôle de Q, K, V et comment les matrices apprennent.

In [8]:
class SelfAttention(nn.Module):
    """
    Module de Self-Attention.
    
    Projette l'input x vers Q, K, V puis applique l'attention.
    """
    
    def __init__(self, embed_dim):
        """
        Args:
            embed_dim: Dimension des embeddings d'entrée
        """
        super().__init__()
        self.embed_dim = embed_dim
        
        # CORRECTION 1: Créer 3 couches linéaires pour projeter vers Q, K, V
        self.W_q = nn.Linear(embed_dim, embed_dim)
        self.W_k = nn.Linear(embed_dim, embed_dim)
        self.W_v = nn.Linear(embed_dim, embed_dim)
    
    def forward(self, x):
        """
        Args:
            x: Embeddings, shape (batch, seq_len, embed_dim)
        
        Returns:
            output: Résultat de l'attention
            attention_weights: Poids d'attention
        """
        # CORRECTION 2: Projeter x vers Q, K, V
        Q = self.W_q(x)
        K = self.W_k(x)
        V = self.W_v(x)
        
        # CORRECTION 3: Appliquer la fonction scaled_dot_product_attention
        output, attention_weights = scaled_dot_product_attention(Q, K, V)
        
        return output, attention_weights

In [9]:
# Test du module
embed_dim = 32
batch_size = 2
seq_len = 5

attention_layer = SelfAttention(embed_dim)
x = torch.randn(batch_size, seq_len, embed_dim)

output, weights = attention_layer(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")   # Attendu: (2, 5, 32)
print(f"Weights shape: {weights.shape}") # Attendu: (2, 5, 5)

# Compter les paramètres
n_params = sum(p.numel() for p in attention_layer.parameters())
print(f"\nNombre de paramètres: {n_params:,}")

Input shape: torch.Size([2, 5, 32])
Output shape: torch.Size([2, 5, 32])
Weights shape: torch.Size([2, 5, 5])

Nombre de paramètres: 3,168


---

## 4. Visualisation sur CamemBERT

Maintenant qu'on a compris et implémenté l'attention, regardons ce que ça donne sur un modèle **réellement entraîné**.

On utilise **CamemBERT**, le modèle français qu'on a découvert au TP précédent.

### Tokens spéciaux CamemBERT

| Token | Rôle |
|-------|------|
| **\<s\>** | Début de phrase (équivalent [CLS]) |
| **\</s\>** | Fin de phrase (équivalent [SEP]) |

### Choix de la couche et de la tête

CamemBERT a **12 couches** et **12 têtes par couche** = 144 matrices d'attention différentes !

Toutes ne sont pas intéressantes à visualiser. On va explorer différentes têtes pour voir lesquelles capturent la **coréférence** (le lien entre un pronom et son antécédent).

### Plan de cette section

1. **Phrase standard** : observer la coréférence sur du vocabulaire courant
2. **Phrase Pokémon** : observer les limites avec du vocabulaire hors-domaine

In [10]:
from transformers import CamembertModel, CamembertTokenizer

# Charger CamemBERT
print("Chargement de CamemBERT...")
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertModel.from_pretrained("camembert-base", output_attentions=True)
model.eval()

print(f"Modèle chargé !")
print(f"  - Couches : {model.config.num_hidden_layers}")
print(f"  - Têtes par couche : {model.config.num_attention_heads}")

Chargement de CamemBERT...
Modèle chargé !
  - Couches : 12
  - Têtes par couche : 12


In [ ]:
# 1. PHRASE STANDARD - Coréférence claire
phrase_standard = "Le chat dort sur le canapé car il est fatigué"

inputs = tokenizer(phrase_standard, return_tensors="pt")
tokens_standard = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

with torch.no_grad():
    outputs = model(**inputs)

attentions_standard = outputs.attentions

print("=== PHRASE STANDARD ===")
print(f"Phrase: {phrase_standard}")
print(f"Tokens: {tokens_standard}")
print("\nObservation : chaque mot = 1 token (vocabulaire courant)")

### 4.1 Visualisation de la coréférence (phrase standard)

Avec une phrase standard, le pronom "il" devrait pointer vers "chat".

In [ ]:
def plot_attention(attention_matrix, tokens, title="Attention"):
    """Affiche une matrice d'attention avec matplotlib."""
    plt.figure(figsize=(10, 8))
    plt.imshow(attention_matrix, cmap='Blues')
    plt.xticks(range(len(tokens)), tokens, rotation=45, ha='right')
    plt.yticks(range(len(tokens)), tokens)
    plt.xlabel("Tokens regardés (Keys)")
    plt.ylabel("Tokens qui regardent (Queries)")
    plt.title(title)
    plt.colorbar(label="Poids d'attention")
    
    # Ajouter les valeurs
    for i in range(len(tokens)):
        for j in range(len(tokens)):
            val = attention_matrix[i, j]
            color = 'white' if val > 0.3 else 'black'
            plt.text(j, i, f'{val:.2f}', ha='center', va='center',
                    color=color, fontsize=7)
    plt.tight_layout()
    plt.show()

In [1]:
# Visualiser l'attention sur la phrase standard
layer = 7  # Couche 8 (0-indexed)
head = 9   # Tête 10 (0-indexed)

attention_matrix = attentions_standard[layer][0, head].numpy()
plot_attention(attention_matrix, tokens_standard, f"Phrase standard - Couche {layer+1}, Tête {head+1}")

NameError: name 'attentions_standard' is not defined

In [ ]:
# Que regarde le pronom "il" dans la phrase standard ?
il_index = None
for i, t in enumerate(tokens_standard):
    if t == "▁il":
        il_index = i
        break

if il_index:
    print(f"Attention de 'il' (Couche {layer+1}, Tête {head+1}) :")
    print("-" * 50)
    
    for i, (token, weight) in enumerate(zip(tokens_standard, attention_matrix[il_index])):
        bar = "*" * int(weight * 30)
        highlight = " <-- antécédent !" if "chat" in token.lower() else ""
        print(f"  {token:15} {weight:.3f} {bar}{highlight}")
    
    print("\n'il' devrait regarder principalement 'chat' (coréférence)")
else:
    print("Token 'il' non trouvé")

In [2]:
# Recherche automatique des meilleures têtes pour la coréférence "il" -> "chat"
il_idx = tokens_standard.index("▁il") if "▁il" in tokens_standard else None
chat_idx = tokens_standard.index("▁chat") if "▁chat" in tokens_standard else None

if il_idx and chat_idx:
    print("Recherche des têtes qui capturent le mieux 'il' -> 'chat'...")
    print("=" * 60)
    
    # Parcourir toutes les couches et têtes
    scores = []
    for layer_i in range(len(attentions_standard)):
        for head_i in range(attentions_standard[layer_i].shape[1]):
            weight = attentions_standard[layer_i][0, head_i, il_idx, chat_idx].item()
            scores.append((weight, layer_i, head_i))
    
    # Trier par poids décroissant
    scores.sort(reverse=True)
    
    print(f"\nTop 5 des têtes où 'il' regarde le plus 'chat' :")
    print("-" * 60)
    for weight, layer_i, head_i in scores[:5]:
        print(f"  Couche {layer_i+1:2d}, Tête {head_i+1:2d} : poids = {weight:.3f}")
    
    print(f"\nPour visualiser la meilleure : layer={scores[0][1]}, head={scores[0][2]}")
else:
    print("Tokens 'il' ou 'chat' non trouvés")

NameError: name 'tokens_standard' is not defined

### 4.2 Limites avec le vocabulaire Pokémon

Essayons maintenant avec notre corpus Pokémon. Les noms comme "Pikachu" ou "Dracaufeu" sont **hors du vocabulaire** de CamemBERT (entraîné sur du français standard).

**Conséquence** : ces mots sont découpés en sous-tokens, ce qui rend la coréférence plus difficile à capturer.

In [3]:
# 2. PHRASE POKÉMON - Vocabulaire hors-domaine
phrase_pokemon = "Pikachu a utilisé Tonnerre sur Dracaufeu car il était très efficace"

inputs_pokemon = tokenizer(phrase_pokemon, return_tensors="pt")
tokens_pokemon = tokenizer.convert_ids_to_tokens(inputs_pokemon["input_ids"][0])

with torch.no_grad():
    outputs_pokemon = model(**inputs_pokemon)

attentions_pokemon = outputs_pokemon.attentions

print("=== PHRASE POKÉMON ===")
print(f"Phrase: {phrase_pokemon}")
print(f"Tokens: {tokens_pokemon}")
print(f"\nObservation : 'Pikachu' et 'Dracaufeu' sont découpés en sous-tokens !")
print("Le modèle n'a jamais vu ces mots pendant son entraînement.")

NameError: name 'tokenizer' is not defined

In [4]:
# Visualiser l'attention sur la phrase Pokémon
attention_matrix_pokemon = attentions_pokemon[layer][0, head].numpy()
plot_attention(attention_matrix_pokemon, tokens_pokemon, f"Phrase Pokémon - Couche {layer+1}, Tête {head+1}")

NameError: name 'attentions_pokemon' is not defined

In [6]:
# Que regarde "il" dans la phrase Pokémon ?
il_index_pokemon = None
for i, t in enumerate(tokens_pokemon):
    if "il" in t.lower():
        il_index_pokemon = i
        break

if il_index_pokemon:
    print(f"Attention de 'il' (Couche {layer+1}, Tête {head+1}) :")
    print("-" * 50)
    
    for i, (token, weight) in enumerate(zip(tokens_pokemon, attention_matrix_pokemon[il_index_pokemon])):
        bar = "*" * int(weight * 30)
        print(f"  {token:15} {weight:.3f} {bar}")
    
    print("\nLa coréférence est moins claire : 'il' peut référer à Pikachu, Tonnerre, ou Dracaufeu.")
    print("De plus, ces entités sont fragmentées en sous-tokens.")
else:
    print("Token 'il' non trouvé")

NameError: name 'tokens_pokemon' is not defined

In [ ]:
# Comparaison : utiliser les MÊMES têtes (trouvées sur phrase standard) sur la phrase Pokémon
# Hypothèse : si une tête capture la coréférence, elle devrait le faire quelle que soit la phrase

print("Comparaison des meilleures têtes de coréférence sur les deux phrases")
print("=" * 70)
print("(Têtes trouvées sur phrase standard, appliquées sur phrase Pokémon)\n")

il_idx_poke = tokens_pokemon.index("▁il") if "▁il" in tokens_pokemon else None

if il_idx_poke and 'scores' in dir():
    for weight_std, layer_i, head_i in scores[:3]:  # Top 3 têtes
        print(f"Couche {layer_i+1}, Tête {head_i+1} (poids 'il'→'chat' sur standard = {weight_std:.3f}) :")
        
        # Sur phrase Pokémon : où regarde "il" ?
        weights_poke = attentions_pokemon[layer_i][0, head_i, il_idx_poke].numpy()
        top_indices = weights_poke.argsort()[-5:][::-1]
        
        print(f"    → Sur phrase Pokémon, 'il' regarde :")
        for idx in top_indices:
            print(f"        {tokens_pokemon[idx]:15} : {weights_poke[idx]:.3f}")
        print()
    
    print("-" * 70)
    print("Observation : les mêmes têtes qui capturent 'il'→'chat' sur la phrase")
    print("standard ne trouvent pas nécessairement la bonne coréférence sur la")
    print("phrase Pokémon (vocabulaire fragmenté + ambiguïté sémantique).")
else:
    print("Exécutez d'abord les cellules précédentes (scores et tokens_pokemon)")

### 4.3 Comment améliorer ?

Le modèle a du mal avec le vocabulaire Pokémon car :
1. **Tokenization fragmentée** : "Pikachu" = plusieurs sous-tokens
2. **Embeddings non spécialisés** : le modèle n'a jamais appris ces concepts

**Solutions possibles** :
- **Fine-tuning** : ré-entraîner sur un corpus Pokémon pour adapter les embeddings
- **Enrichir le tokenizer** : ajouter "Pikachu", "Dracaufeu" comme tokens uniques avec `tokenizer.add_tokens()`, puis fine-tuner

Ces techniques seront abordées plus tard.

### 4.4 Visualisation interactive avec BertViz

**BertViz** est un outil spécialisé pour visualiser l'attention des Transformers.

- **head_view** : voir les connexions d'attention pour une couche (carrés colorés = têtes)
- **model_view** : vue globale de toutes les couches et têtes

In [7]:
from bertviz import head_view, model_view

# BertViz sur la phrase standard (coréférence plus visible)
print("head_view sur la phrase standard")
print("Cliquez sur une tête pour voir ses connexions d'attention")
print("="*50)

head_view(attentions_standard, tokens_standard)

head_view sur la phrase standard
Cliquez sur une tête pour voir ses connexions d'attention


NameError: name 'attentions_standard' is not defined

In [ ]:
# BertViz sur la phrase Pokémon (pour comparer)
print("head_view sur la phrase Pokémon")
print("Comparez avec la phrase standard : la coréférence est-elle aussi claire ?")
print("="*50)

head_view(attentions_pokemon, tokens_pokemon)

---

## 5. Pourquoi Multi-Head Attention ?

### Le problème avec une seule tête

Une seule tête d'attention calcule **une** représentation des relations entre mots.

Mais dans une phrase, il y a **plusieurs types de relations** :
- Relations **syntaxiques** (sujet-verbe)
- Relations **sémantiques** (sens, coréférence)
- Relations de **proximité** (mots voisins)
- etc.

### La solution : plusieurs têtes

Chaque tête peut apprendre à détecter un type de relation différent !

**Attention** : En pratique, une tête ne correspond pas forcément à un type de relation précis. Une relation peut être portée par une combinaison de têtes, et une tête peut capturer plusieurs types de patterns. C'est ce que le modèle apprend pendant l'entraînement.

**Analogie** : C'est comme avoir plusieurs experts qui analysent une phrase sous différents angles, puis combinent leurs analyses.

### Exemple : comparons deux têtes de CamemBERT

In [ ]:
# Comparaison de 2 têtes sur la phrase standard
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Tête des couches profondes
head_deep = attentions_standard[9][0, 5].numpy()  # Couche 10, Tête 6

# Tête des premières couches (attention locale)
head_local = attentions_standard[1][0, 0].numpy()  # Couche 2, Tête 1

for idx, (attn, title) in enumerate([
    (head_deep, "Couche 10, Tête 6\n(couches profondes)"),
    (head_local, "Couche 2, Tête 1\n(attention locale)")
]):
    ax = axes[idx]
    im = ax.imshow(attn, cmap='Blues')
    ax.set_xticks(range(len(tokens_standard)))
    ax.set_xticklabels(tokens_standard, rotation=45, ha='right', fontsize=9)
    ax.set_yticks(range(len(tokens_standard)))
    ax.set_yticklabels(tokens_standard, fontsize=9)
    ax.set_title(title, fontsize=12)
    plt.colorbar(im, ax=ax)

plt.suptitle("Deux têtes capturent des patterns différents", fontsize=13, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Que regarde "il" selon chaque tête ? (phrase standard)
il_idx = tokens_standard.index("▁il") if "▁il" in tokens_standard else None

if il_idx:
    print(f"Que regarde 'il' selon chaque tête ?")
    print("=" * 50)
    
    print(f"Tête profonde (C10-T6) : ", end="")
    for i, w in enumerate(head_deep[il_idx]):
        if w > 0.08:
            print(f"{tokens_standard[i]}({w:.2f}) ", end="")
    print()
    
    print(f"Tête locale (C2-T1) :    ", end="")
    for i, w in enumerate(head_local[il_idx]):
        if w > 0.08:
            print(f"{tokens_standard[i]}({w:.2f}) ", end="")
    print()
    
    print("\nLes couches profondes capturent mieux la coréférence (il -> chat)")

---

## 6. Architecture Multi-Head Attention

### Schéma

```
Input (seq_len, embed_dim)
        |
   +----+----+--------+--------+
   |         |        |        |
   v         v        v        v
 Head 1   Head 2   Head 3   Head 4
   |         |        |        |
   +----+----+--------+--------+
        |
    Concat
        |
   Linear (W_o)
        |
Output (seq_len, embed_dim)
```

### Dimensions

| Paramètre | Exemple | Description |
|-----------|---------|-------------|
| embed_dim | 512 | Dimension des embeddings |
| num_heads | 8 | Nombre de têtes |
| d_k | 64 | Dimension par tête = embed_dim / num_heads |

Chaque tête travaille avec une **dimension réduite** ($d_k$), puis on **concatène** les résultats.

### Pourquoi W_o après le concat ?

Après concaténation, on a un vecteur de dimension `embed_dim` (4 têtes x 64 = 512).

La matrice **W_o** (output projection) permet de :
1. **Mélanger** les informations des différentes têtes
2. **Apprendre** comment combiner leurs "points de vue"

Sans W_o, les têtes resteraient indépendantes. Avec W_o, le modèle peut apprendre des combinaisons utiles.

## 7. Exercice 3 : split_heads (Multi-Head)

La première étape du Multi-Head est de **séparer les têtes**.

On transforme `(batch, seq_len, embed_dim)` en `(batch, num_heads, seq_len, d_k)`

In [8]:
# Configuration
batch_size = 2
seq_len = 6
embed_dim = 32
num_heads = 4
d_k = embed_dim // num_heads  # 32 / 4 = 8

print(f"embed_dim: {embed_dim}")
print(f"num_heads: {num_heads}")
print(f"d_k (dim par tête): {d_k}")

# Input
x = torch.randn(batch_size, seq_len, embed_dim)
print(f"\nInput shape: {x.shape}")

embed_dim: 32
num_heads: 4
d_k (dim par tête): 8


NameError: name 'torch' is not defined

In [9]:
def split_heads(x, num_heads):
    """
    Sépare les têtes d'attention.
    
    Reshape: (batch, seq_len, embed_dim) -> (batch, num_heads, seq_len, d_k)
    
    Args:
        x: Tensor de shape (batch, seq_len, embed_dim)
        num_heads: Nombre de têtes
    
    Returns:
        Tensor de shape (batch, num_heads, seq_len, d_k)
    """
    batch_size, seq_len, embed_dim = x.shape
    d_k = embed_dim // num_heads
    
    # CORRECTION:
    # Étape 1: Séparer embed_dim en (num_heads, d_k)
    x = x.view(batch_size, seq_len, num_heads, d_k)
    # Étape 2: Réorganiser pour avoir num_heads en position 1
    x = x.transpose(1, 2)
    
    return x

In [10]:
# Test
x_test = torch.randn(2, 6, 32)  # batch=2, seq=6, embed=32
x_heads = split_heads(x_test, num_heads=4)

print(f"Avant split: {x_test.shape}")   # (2, 6, 32)
print(f"Après split: {x_heads.shape}")  # Attendu: (2, 4, 6, 8)

if x_heads is not None and x_heads.shape == (2, 4, 6, 8):
    print("\nCorrect !")
else:
    print("\nVérifiez votre implémentation")

NameError: name 'torch' is not defined

---

## 8. Récapitulatif

### Ce que nous avons appris aujourd'hui

| Concept | Ce qu'on a fait |
|---------|----------------|
| **Exercice 1** | Fonction `scaled_dot_product_attention()` complète |
| **Exercice 2** | Classe `SelfAttention` avec projections W_q, W_k, W_v |
| **Visualisation** | Explorer l'attention de CamemBERT avec BertViz |
| **Multi-Head** | Comprendre pourquoi plusieurs têtes |
| **Exercice 3** | Fonction `split_heads()` pour séparer les têtes |

### Questions de compréhension

Avant de passer au TP suivant, vérifiez que vous pouvez répondre à ces questions :

1. **Scaling** : Que se passe-t-il si on ne divise pas par sqrt(d_k) ? Pourquoi ?

2. **Softmax** : Pourquoi la somme des poids d'attention fait toujours 1 ?

3. **Paramètres** : Combien de paramètres a une couche `SelfAttention` avec `embed_dim=512` ? (indice : 3 matrices W_q, W_k, W_v)

4. **Multi-Head** : Si on a `embed_dim=512` et `num_heads=8`, quelle est la dimension d_k de chaque tête ?

5. **Q != K** : Pourquoi utilise-t-on des matrices différentes pour Q et K au lieu de faire simplement `x @ x.T` ?

<details>
<summary>Réponses</summary>

1. Les scores deviennent très grands -> softmax donne des poids proches de 0 ou 1 -> gradients instables

2. Les poids d'attention sont obtenus en appliquant softmax aux scores. Par définition, softmax produit des valeurs positives qui somment à 1 (distribution de probabilités).

3. 3 x (512 x 512 + 512) = 3 x 262 656 = **787 968** paramètres (poids + biais)

4. d_k = 512 / 8 = **64**

5. Pour permettre au modèle de capturer d'autres relations que la similarité sémantique (relations syntaxiques, contextuelles, etc.)
</details>

### Prochaine session

On va :
1. Terminer l'implémentation Multi-Head (`concat_heads`, classe complète)
2. Ajouter le **Feed-Forward Network**
3. Comprendre le **masque causal** (GPT vs BERT)
4. Assembler un **bloc Transformer** complet !

---

## 9. Pour aller plus loin (optionnel)

### 9.1 Explorer toutes les têtes d'une couche

In [11]:
# Visualiser 4 têtes différentes de la même couche (phrase standard)
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

heads_to_show = [0, 3, 7, 11]
layer = 8  # Couche 9

for idx, head in enumerate(heads_to_show):
    ax = axes[idx // 2, idx % 2]
    w = attentions_standard[layer][0, head].numpy()
    
    im = ax.imshow(w, cmap='Blues')
    ax.set_xticks(range(len(tokens_standard)))
    ax.set_xticklabels(tokens_standard, rotation=45, ha='right', fontsize=8)
    ax.set_yticks(range(len(tokens_standard)))
    ax.set_yticklabels(tokens_standard, fontsize=8)
    ax.set_title(f"Tête {head + 1}", fontsize=11)
    plt.colorbar(im, ax=ax)

plt.suptitle(f"Différentes têtes de la couche {layer+1} - CamemBERT", fontsize=13)
plt.tight_layout()
plt.show()

NameError: name 'plt' is not defined

In [12]:
# Analyse : que regarde "il" selon chaque tête ?
il_idx = tokens_standard.index("▁il") if "▁il" in tokens_standard else None

if il_idx:
    print(f"Que regarde 'il' selon chaque tête de la couche {layer+1} ?")
    print("=" * 60)
    
    for head in heads_to_show:
        weights = attentions_standard[layer][0, head, il_idx].numpy()
        top_idx = weights.argsort()[-3:][::-1]  # Top 3
        print(f"Tête {head+1:2d}: ", end="")
        for i in top_idx:
            print(f"{tokens_standard[i]} ({weights[i]:.2f})  ", end="")
        print()

NameError: name 'tokens_standard' is not defined

### 9.2 Testez vos propres phrases

Utilisez cette fonction pour analyser n'importe quelle phrase :

In [13]:
def analyser_phrase(phrase):
    """Analyse l'attention de CamemBERT sur une phrase."""
    inputs = tokenizer(phrase, return_tensors="pt")
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    print(f"Phrase: {phrase}")
    print(f"Tokens: {tokens}\n")
    
    return head_view(outputs.attentions, tokens)

# Testez avec vos phrases !
analyser_phrase("Marie a appelé son frère car il était en retard")

NameError: name 'tokenizer' is not defined